Cluster the topics into groups

In [37]:
import pandas as pd
import time #to have today's date
timestr = time.strftime("%Y%m%d")

## Import dataframes

In [26]:
#import dataframe with all topics and the 20 most representatives texts and the 20 most important keywords
#prepared in p_10_dominant_topics_in_texts_qa
df_top_docs_per_topic_keyw = pd.read_csv('.//results//topics_group//qa_topic_keywords_representative_text_'+timestr+'_V01.csv',
    sep=';', decimal=',', index_col=0) #check for most recent version!

In [27]:
#import the dataframe with the exclusive words (low lambda)
#prepared in p_09_analyse_topics_qa
df_topics_keyw_lam = pd.read_csv('.//results//topics_group//qa_topic_original_numbers_keywords_lambda_'+timestr+'_V01.csv',
    sep=';', decimal=',', index_col=0)

In [28]:
#prepared in p_06_set_parameters_LDA_qa_
df_topicnum_grouped_3_kw = pd.read_csv\
        ('.\\results\LDA_set_parameters\parameters_lda\qa_overview_10_topicnum_match_03_keyw_'+timestr+'_V01.csv',
            sep=';', decimal=',', index_col=0)

In [29]:
df_topicnum_grouped_3_kw = df_topicnum_grouped_3_kw[['tnum_22_topic_scores', 'tn_tnum_22']]
df_scores = df_topicnum_grouped_3_kw.drop_duplicates().dropna().reset_index(drop=True)
df_scores = df_scores.rename(columns={'tn_tnum_22': 'ori_topic_num', 'tnum_22_topic_scores':'topic_score_%'})

## merge and export dataframe for manual clustering

In [30]:
#add topic scores to topics and keywords
df_cluster_topics_first = df_topics_keyw_lam.merge(df_scores, how='outer')

In [31]:
#add respresentative documents to each topic
df_cluster_topics = df_top_docs_per_topic_keyw.copy().merge(df_cluster_topics_first, how='outer')

In [32]:
df_cluster_topics['Topic_label']=''
df_cluster_topics['Category']=''
df_cluster_topics['Topic_Perc_Contrib'] = round(df_cluster_topics['Topic_Perc_Contrib'], 2)

In [34]:
df_cluster_topics = df_cluster_topics[['ori_topic_num', 'Topic_label', 'Category', 'topic_score_%', 'keywords',\
            'lambda_0', 'Topic_Perc_Contrib', 'Text']]

In [35]:
#check: each topic has 20 documents?
for x in set(df_cluster_topics['ori_topic_num']):
    df_x_len = df_cluster_topics[df_cluster_topics['ori_topic_num'] == x]
    if len(df_x_len) != 20:
        print('topic '+ str(x) + ' has ' + str(len(df_x_len)) + ' documents')

In [36]:
#re-arrange:
df_cluster_topics = df_cluster_topics.sort_values(['ori_topic_num', 'Topic_Perc_Contrib'], \
                                ascending=[True, False]).reset_index(drop=True)

In [37]:
for x in ['ori_topic_num', 'topic_score_%', 'keywords', 'lambda_0']:
    df_cluster_topics[x] = df_cluster_topics[x].drop_duplicates()

In [38]:
df_cluster_topics['topic_score_%'] = round(df_cluster_topics['topic_score_%'], 1)

In [43]:
df_cluster_topics.to_excel('.//results//topics_group//topics_label_category_manually_t22_'+timestr+'_V01.xlsx')

## import dataframe with clustered topics

In [2]:
#P: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe1 in position 4467: invalid continuation byte
#S: check encoding with chardet
# to get the best encoding
import chardet
filename = './/results//topics_group//manually_labelled//topics_label_category_MERGED_'+timestr+'_short_V01.csv'
with open(filename, 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [3]:
#import with clustered topics:
df_topics_categories_imp = pd.read_csv('.//results//topics_group//manually_labelled//topics_label_category_MERGED_'+timestr+'_short_V01.csv',
    sep=';', decimal=',', encoding='ISO-8859-1')

## renumber topics, according to categories

In [12]:
df_topics_categories_imp.rename(columns={'Final_Topic_label':'Topic_label', 'Final_Category':'Category_label'}, inplace=True)

In [15]:
#topics should have numbers according to their categories
#this makes it nicer to show later in the graphs
df_topics_categories_imp = df_topics_categories_imp.copy()[['ori_topic_num', 'topic_score_%', 'Topic_label', 'Category_label', 'keywords', \
                'lambda_0']].dropna().reset_index(drop=True)

In [21]:
list(set(df_topics_categories_imp['Category_label']))

['Policies',
 'Energy governance & markets',
 'Actors & interests',
 'Electric power system']

In [28]:
cat_list_alpha = ['Actors & interests', 'Electric power system', 'Energy governance & markets', 'Policies'] 

In [29]:
#add numbers to the categories
df_cat_num = pd.DataFrame(data={'Category_num':range(1,len(set(df_topics_categories_imp['Category_label']))+1),\
                                'Category_label':cat_list_alpha})

df_topics_categories = df_topics_categories_imp.merge(df_cat_num)

In [31]:
#sort by categories and number the topics again
df_topics_categories_num = df_topics_categories.sort_values('Category_num').reset_index(drop=True)
df_topics_categories_num['Topic_num'] = range(1, len(df_topics_categories_num)+1)

In [32]:
df_topics_categories_num.columns

Index(['ori_topic_num', 'topic_score_%', 'Topic_label', 'Category_label',
       'keywords', 'lambda_0', 'Category_num', 'Topic_num'],
      dtype='object')

In [34]:
df_topics_categories_num = df_topics_categories_num[['Category_num', 'Category_label', \
                    'Topic_num', 'Topic_label', 'topic_score_%', 'keywords', 'lambda_0', 'ori_topic_num']]

## export dataframe with topics and categories

In [35]:
#dataframe with the topics, numbers, names, categories and keywords

In [36]:
df_topics_categories_num.to_csv\
('.//results//master_dfs//category_numname_topics_numname_orinew_keywords_lambda_'+timestr+'_V01.csv',
    sep=';', decimal=',')